In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
from image_utils import *

%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research

!pip install tensorflow-graphics

from __future__ import absolute_import, division, print_function
from absl import logging
import os
# import image_utils
import cv2
import re
import time
import glob
from matplotlib import pyplot as plt
import pickle
import numpy as np
from PIL import Image
import pandas as pd

import tensorflow as tf
from depth_from_video_in_the_wild import model

# https://github.com/google-research/google-research/tree/master/depth_from_video_in_the_wild

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research
     |████████████████████████████████| 245kB 2.8MB/s 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Feature and Image Paths:

In [0]:
cropped_img_path = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/'
optimal_crop_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/' 
# path to depth_from_the_wild outputs
path_odom = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild/odometry_trajectory_outputs/'
path_depth = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/distance_maps/'

root = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents'
# path
train_path = root + '/dataset/features/training/' 
test_path = root +  '/dataset/features/testing/' 

###
batch_str = 'batch_045.npz'
mode = 'training'
vid_num = 2
###

num1 = (126, 46)
org_bb = np.load(train_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
resized_bb = np.load(optimal_crop_path+'training/'+batch_str)['new_det']
# track_ids = np.load(path_track_ids+'training/'+batch_str)
frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'],mode, num1[0])
odom_complete_paths = retrieve_dir_custom(path_odom, org_bb, 'training', num1[0], split_type=1)
depth_complete_paths = retrieve_dir_custom(path_depth, org_bb, 'training', num1[0])

depth_np = np.load(depth_complete_paths[vid_num]) # ['distance_map']
odom_np = np.load(odom_complete_paths[vid_num]) # ['rot', 'trans', 'instrinsic_matrix', 'orientation', 'position']

cur_vid_folder = re.split('/', frame_paths[vid_num])[-3]+'_'+re.split('/', frame_paths[vid_num])[-2]
cur_img_path = cropped_img_path+'/'+'training'+'/' + cur_vid_folder
# create list
folders = sorted(os.listdir(cur_img_path))
img_list = [cur_img_path +'/' + f for f in folders if 'png' in f and 'seg' not in f]

In [0]:
new_save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/'
# path_glob = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_2/'

num1 = (126, 46)

def retrieve_global_coord(global_df_path, feat_path, batch_str, num_samp, mode='training'):
  glob_feat = np.zeros((10,100, 20, 4))
  org_bb = np.load(feat_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
  frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'], mode, num_samp)
  global_df_paths = retrieve_dir_custom(new_save_path, org_bb, mode, num_samp, split_type=2)
 
  for batch_num in range(10):
    cur_df_path = global_df_paths[batch_num]
    # add try except functionallity in case df doesn't exisit
    try:
      trajectory_df = pd.read_pickle(cur_df_path)
      time_stamps = np.linspace(0,5,100) # use time_stamps to retrieve the correct index
      frames = trajectory_df.groupby('TIMESTAMPS')

      for group_name, group_data in frames:
        time_ind = np.argwhere(time_stamps == group_name)[0][0] 
        cur_x, cur_y = group_data['X'], group_data['Z']
        v_x, v_y = group_data['v_x'], group_data['v_y']
        glob_feat[batch_num, time_ind,0:len(cur_x),0] = cur_x
        glob_feat[batch_num, time_ind,0:len(cur_x),1] = cur_y
        glob_feat[batch_num, time_ind,0:len(cur_x),2] = v_x
        glob_feat[batch_num, time_ind,0:len(cur_x),3] = v_y
        # 4 dimensions
        # 4 velocity dimensions
    except:
      print("Dataframe doesn't exist: " + str(cur_df_path))


  # world_coordinate modification
  # glob_feat = take_out_nans(glob_feat)
  # glob_feat[glob_feat == np.inf] = 0
  # glob_feat[glob_feat*(-1) == np.inf] = 0
  # glob_feat = glob_feat*0.1  ## change/normalize the global features

  return glob_feat 

def take_out_nans(phys_feat):
    if np.sum(np.isnan(phys_feat)) > 0:
      nan_indices = np.argwhere(np.isnan(phys_feat))
      for k in range(nan_indices.shape[0]):
        phys_feat[nan_indices[k,0],nan_indices[k,1],nan_indices[k,2],nan_indices[k,3]] = 0
    return phys_feat

In [0]:
## turn visualization into function
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.image as mpimg

## resized_bb, cropped_img_path, vid_num, batch, mode = 'training'
batch_str = 'batch_022.npz'
mode = 'training'
vid_num = 2
# feat_path = '../../Anticipating-Accidents/dataset/features/'+a1+'/'
# det_path = optimal_crop_path

num1 = (126, 46)
org_bb = np.load(train_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
resized_bb = np.load(optimal_crop_path+mode+'/'+batch_str)['new_det']
# track_ids = np.load(path_track_ids+'training/'+batch_str)
frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'],mode, num1[0])

df_paths = retrieve_dir_custom(new_save_path, org_bb, 'training', num1[0], split_type=2)
phys_feat = retrieve_global_coord(df_paths, train_path, batch_str, num1[0], mode='training')

# batch_num = 0
# frame = 18
# print(phys_feat[batch_num, frame, :, :])
# print(phys_feat.shape)

Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/positive_000073.pkl
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/negative_000128.pkl
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/positive_000055.pkl
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/positive_000056.pkl
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/negative_000131.pkl
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/positive_000091.pkl
Dataframe doesn't exist: /content/gdrive/My Dr

In [0]:
def show_agents_visualization(resized_bb, cropped_img_path, vid_num, batch_str, frame_paths, trajectory_df, k_width = 12, mode= 'training'):
  cur_vid_folder = re.split('/', frame_paths[vid_num])[-3]+'_'+re.split('/', frame_paths[vid_num])[-2]
  cur_img_path = cropped_img_path+'/'+mode+'/' + cur_vid_folder
  folders = sorted(os.listdir(cur_img_path))
  img_list = [cur_img_path +'/' + f for f in folders if 'png' in f and 'seg' not in f]
  bboxes = resized_bb[vid_num]
  print(bboxes.shape)
  crop_type = np.max(np.max(bboxes[:,:,-2]))
  str_path = cropped_img_path+mode
  folders = sorted(os.listdir(str_path))
  # rgb_path = '../../Anticipating-Accidents/dataset/videos/frames/'+a1 +'/'+a2 +'/'+cur_file + '/'
  # video_path =  '../../Anticipating-Accidents/dataset/videos/'+a1 +'/'+a2 +'/'+cur_file +'.mp4'
  fig, ax = plt.subplots()
  max_agent_ind = np.max(bboxes[:,:,-1])
  frames = trajectory_df.groupby('Agent_ID')

  for img_ind in range(98):
      cur_img_pth = img_list[img_ind]
      # crop image to dimensions
      # cur_img_pth = rgb_path +'frame'+str(img_ind)+'.jpg' ############ change this name
      frame = cv2.imread(cur_img_pth)
      frame = frame[:, 0:416, :] # crop images, since three are fused together for training purposes
      print(frame.shape)
      new_bboxes = bboxes[img_ind]
      new_bboxes = new_bboxes[~np.all(new_bboxes == 0, axis=-1)] ## take out zero values from new_bboxes  
      print(new_bboxes)
      #############################
      ## Replace with plotting function
      agents_ids = new_bboxes[:,-1]

      ax = _add_traj(frames, img_ind, ax, k_width = k_width)

  #     for i, txt in enumerate(agents_ids):
  #       ax.annotate(txt, (pf[i,0]*80, pf[i,1]*80))
  #     ax.scatter(pf[:,0]*80,pf[:,1]*80)
      ############################################################################
      fig.canvas.draw()
      img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8,sep='')
      img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
      img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
      # a11,a22,a33 = img.shape
      # ratio = frame.shape[0]/a11
      # w1 = int(a22*2/ratio)
      # h1 = int(a11*2/ratio)
      # re_img = cv2.resize(img,(w1,h1), interpolation = cv2.INTER_AREA)   
      # overlay = cv2.addWeighted(frame[0:h1,-w1-1:-1],0.1,re_img,0.9,0)
      # frame[0:h1,-w1-1:-1] = overlay
      ############################################################################
      font = cv2.FONT_HERSHEY_SIMPLEX # change font type?
  #     frame = cv2.imread(cur_img_pth)
      for num_box in range(new_bboxes.shape[0]):
          # print(img_ind)
          # print(new_bboxes[num_box,0],new_bboxes[num_box,1],new_bboxes[num_box,2],new_bboxes[num_box,3] )
          cv2.rectangle(frame,(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])),(int(new_bboxes[num_box,2]),int(new_bboxes[num_box,3])),(0,255,0),1)
          # add agent ids in top right-hand corner
          cv2.putText(frame,str(new_bboxes[num_box,-1]),(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])), font, 0.3,(0,0,255),1,cv2.LINE_AA) ## put physical parameters here

      top_left_label = str(img_ind) + ': ' + str(crop_type) +', ' + cur_vid_folder
      cv2.putText(frame,top_left_label ,(10,30), font, 0.25,(255,255,255),1)
      # cv2_imshow(frame)
      plt_img = cv2.resize(img,(400,200))
      plt_img = cv2.hconcat([plt_img, plt_img])
      frame_resized = cv2.resize(frame, (800, 200))
      cv2_imshow(cv2.vconcat([plt_img, frame_resized]))
  #         plt.show()


def _add_traj(frames, k, ax, k_width = 12):
    #trajectory_df = obtain_trajectory_data(bounding_box, depth_complete_paths, odom_complete_paths, vid,dist_coeff = dist_coeff,intr_type = 'med', num_frames = 99)

    time_stamps = np.linspace(0,5,100)
    min_x, max_x = np.min(np.min(frames['X'])), np.max(np.max(frames['X']))
    min_z, max_z = np.min(np.min(frames['Z'])), np.max(np.max(frames['Z']))
    ax.clear()
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_z, max_z)
    ax.set_title('Frame '+ str(k))

    for group_name, group_data in frames:
      past_k = k - k_width
      if past_k < 0:
        past_k = 0
      mask = (group_data['TIMESTAMPS'].values> time_stamps[past_k])*(group_data['TIMESTAMPS'].values < time_stamps[k])
      true_indices = [i for i, x in enumerate(mask) if x]
      if len(true_indices)>0:
        new_mask = mask
        new_mask[true_indices] = False
        new_mask[true_indices[-1]] = True
      else:
        new_mask = mask

      if group_name != 0:
        cor_x = group_data["X"][mask].values
        cor_y = group_data["Z"][mask].values

        ax.plot(
        cor_x,
        cor_y,
        ".")
        if len(true_indices)>0:
          p_x = np.asscalar(group_data["X"][new_mask].values)
          p_y = np.asscalar(group_data["Z"][new_mask].values)
          v_x = np.asscalar(group_data["v_x"][new_mask].values)
          v_y = np.asscalar(group_data["v_y"][new_mask].values)
          print('Inputs:')
          print(p_x, p_y,v_x, v_y)
          ax.arrow(
          p_x,
          p_y,
          v_x,
          v_y)
      else:
        ego_x = group_data["X"][0:k].values
        ego_y = group_data["Z"][0:k].values
        ax.plot(
        ego_x,
        ego_y,
        "-") 
    return ax


def full_visualization(det, vid_num, batch_str, frame_paths, mode= 'training'):
  resized_bb = det
  cur_img_path = frame_paths[vid_num]
  print(cur_img_path)
  img_list = sort_dir(os.listdir(cur_img_path))
  print(img_list)
  bboxes = resized_bb[vid_num]
  crop_type = np.max(np.max(bboxes[:,:,-2]))
  # str_path = cur_img_path
  # folders = sorted(os.listdir(str_path))
  # rgb_path = '../../Anticipating-Accidents/dataset/videos/frames/'+a1 +'/'+a2 +'/'+cur_file + '/'
  # video_path =  '../../Anticipating-Accidents/dataset/videos/'+a1 +'/'+a2 +'/'+cur_file +'.mp4'

  for img_ind in range(98):
      cur_img_pth = cur_img_path+img_list[img_ind]
      print(cur_img_path)
      frame = cv2.imread(cur_img_pth)
      new_bboxes = bboxes[img_ind]
      new_bboxes = new_bboxes[~np.all(new_bboxes == 0, axis=1)] ## take out zero values from new_bboxes  
      agents_ids = new_bboxes[:,-1]

      # a11,a22,a33 = img.shape
      # ratio = frame.shape[0]/a11
      # w1 = int(a22*2/ratio)
      # h1 = int(a11*2/ratio)
      # re_img = cv2.resize(img,(w1,h1), interpolation = cv2.INTER_AREA)   
      # overlay = cv2.addWeighted(frame[0:h1,-w1-1:-1],0.1,re_img,0.9,0)
      # frame[0:h1,-w1-1:-1] = overlay
      ############################################################################
      font = cv2.FONT_HERSHEY_SIMPLEX # change font type?
  #     frame = cv2.imread(cur_img_pth)
      for num_box in range(new_bboxes.shape[0]):
          # print(img_ind)
          # print(new_bboxes[num_box,0],new_bboxes[num_box,1],new_bboxes[num_box,2],new_bboxes[num_box,3] )
          cv2.rectangle(frame,(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])),(int(new_bboxes[num_box,2]),int(new_bboxes[num_box,3])),(0,255,0),2)
          # add agent ids in top right-hand corner
          cv2.putText(frame,str(new_bboxes[num_box,-1]),(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])), font, 0.5,(0,0,255),2,cv2.LINE_AA) ## put physical parameters here

      top_left_label = str(img_ind) + ': ' + str(crop_type) +', ' + cur_vid_folder
      cv2.putText(frame,top_left_label ,(10,30), font, 0.5,(255,255,255),2)
      # cv2_imshow(frame)
      # plt_img = cv2.resize(img,(400,200))
      # plt_img = cv2.hconcat([plt_img, plt_img])
      frame_resized = cv2.resize(frame, (800, 400))
      cv2_imshow(frame_resized)

In [0]:
## resized_bb, cropped_img_path, vid_num, batch, mode = 'training'
# mode = 'training'
# feat_path = '../../Anticipating-Accidents/dataset/features/'+a1+'/'
# det_path = optimal_crop_path
# batch_str = 'batch_026.npz'
# mode = 'training'
# vid_num = 1

num1 = (126, 46)
# org_bb = np.load(train_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
# frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'],mode, num1[0])
# pkl_str = re.split('/',frame_paths[vid_num])[-3]+'_'+re.split('/',frame_paths[vid_num])[-2]+'.pkl'
save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1/'
new_save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1v/'

# file = open(save_path+pkl_str, 'rb')
# trajectory_df = pickle.load(file)
# file.close()

getting average velocity functiokn

In [0]:
def _threshold_mask(x, thr_x):    
  return (x<(np.median(x)+thr_x))*(x>(np.median(x)-thr_x))
  
def average_velocity(k, k_width, thr, frames, df):
  time_stamps = np.linspace(0,5,100)

  for group_name, group_data in frames:
        past_k = k - k_width
        if past_k < 0:
          past_k = 0
        mask = (group_data['TIMESTAMPS'].values>= time_stamps[past_k])*(group_data['TIMESTAMPS'].values <= time_stamps[k])
        # check if mask only has one time point
        if np.sum(mask) >= 2:
          cor_x, cor_z, time_steps = group_data["X"][mask].values, group_data["Z"][mask].values, group_data['TIMESTAMPS'][mask].values
          c_bb0, c_bb1, c_bb2, c_bb3 = group_data["b0"][mask].values, group_data["b1"][mask].values, group_data["b2"][mask].values, group_data["b3"][mask].values

          thr_mask = _threshold_mask(cor_x, thr)*_threshold_mask(cor_z, thr)
          if(np.sum(thr_mask)<2):
            continue
          cor_x, cor_z, time_steps = cor_x[thr_mask], cor_z[thr_mask], time_steps[thr_mask]

          ind_1 = np.arange(0,np.sum(thr_mask)-1)
          ind_2 = ind_1+1

          d_t = time_steps[ind_2] - time_steps[ind_1]
          # d_t[d_t == 0] = 1 ## fixing the zero points
          dv_x = (cor_x[ind_2] - cor_x[ind_1])/d_t*0.050505 ## fixes scaling so that velocity is per frame
          dv_z = (cor_z[ind_2] - cor_z[ind_1])/d_t*0.050505
          dv_bb0 = (c_bb0[ind_2] - c_bb0[ind_1])/d_t*0.050505
          dv_bb1 = (c_bb1[ind_2] - c_bb1[ind_1])/d_t*0.050505
          dv_bb2 = (c_bb2[ind_2] - c_bb2[ind_1])/d_t*0.050505
          dv_bb3 = (c_bb3[ind_2] - c_bb3[ind_1])/d_t*0.050505

          v_x, v_z = np.mean(dv_x), np.mean(dv_z)
          v_bb0, v_bb1, v_bb2, v_bb3 = np.mean(dv_bb0), np.mean(dv_bb1), np.mean(dv_bb2), np.mean(dv_bb3)

          cur_index = group_data["X"][mask].index[-1]
          # print(v_x, v_y)
          trajectory_df.iat[cur_index,-6] = v_x
          trajectory_df.iat[cur_index,-5] = v_z
          trajectory_df.iat[cur_index,-4] = v_bb0
          trajectory_df.iat[cur_index,-3] = v_bb1
          trajectory_df.iat[cur_index,-2] = v_bb2
          trajectory_df.iat[cur_index,-1] = v_bb3

          # print(trajectory_df.head(cur_index+1))
          # print('Current_index!')
          # print(group_data["X"][mask].index[-1])

  # return trajectory_df  
        # print(v_x, v_y)

Save velocity pickle files

In [0]:
#SERENE DECIDED TO ANALYZE THE VELOCITY HERE! 
import math
import statistics
new_save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_5v/'
def isitinfnan(a,b):
  return (math.isnan(a) or math.isnan(b) or math.isinf(a) or math.isinf(b))

In [0]:
def median_filt_velocities(vels, win):
  new_vels = vels
  for i in range(win,len(vels)-win):
    a = []
    for j in range((i-win),i+1):
      if not isitinfnan(vels[j], vels[j]):
        a.append(vels[j])
    new_vels[i] = statistics.median(a)
  return new_vels

In [22]:
file = open(new_save_path+'positive_000606'+'.pkl', 'rb')
trajectory_df = pickle.load(file)
file.close()
# trajectory_df = pd.read_pickle(save_path+cur_name+'.pkl') #????????????????
print(trajectory_df.head())
print(list(trajectory_df.columns.values))
# print(trajectory_df['Agent_ID'])

# x_mses = []
# y_mses = []
# bb0_mses = []
# bb1_mses = []
# bb2_mses=[]
# bb3_mses=[]

def find_MSE_x_y_from_1_frame_ago(window = 1):

  x_diffs = []
  y_diffs = []
  bb0_diffs = []
  bb1_diffs = []
  bb2_diffs=[]
  bb3_diffs=[]
  # WAS CONSIDERING DIVIDING UP POS AND NEG ANALAYSES, BUT NEED TRACKING TO WORK WELL REGARDLESS OF THE CASE
  for filename in os.listdir(new_save_path):
      file = open(new_save_path+filename, 'rb')
      traj_df = pickle.load(file)
      file.close()
      times = traj_df['TIMESTAMPS']
      xs = traj_df['X']
      ys = traj_df['Y']
      zs = traj_df['Z']
      b0s = traj_df['b0']
      b1s = traj_df['b1']
      b2s = traj_df['b2']
      b3s = traj_df['b3']
      x_vels = traj_df['v_x']
      y_vels = traj_df['v_y']
      bb0_vels = traj_df['v_bb0']
      bb1_vels = traj_df['v_bb1']
      bb2_vels = traj_df['v_bb2']
      bb3_vels = traj_df['v_bb3']

      #TESTING CHANGE HERE!
      x_vels = median_filt_velocities(x_vels, window)
      y_vels = median_filt_velocities(y_vels, window)

      
      all_ids = traj_df['Agent_ID']
      # NOTE: cannot sort by unique agent IDs, makes more sense to go chronologically
      unique_times, unique_pos = np.unique(times, return_index=True)

      np.append(unique_pos,len(unique_pos)-1) 

      for p in range(2,len(unique_pos)-1):
        for curr in range(unique_pos[p],unique_pos[p+1]): 
          curr_id = all_ids[curr]
          for prev in range(unique_pos[p-1],unique_pos[p]):
              if all_ids[prev] == curr_id and xs[prev] >0and ys[prev]>0 and b0s[prev]>0:
                #CALCULATE ESTIMATED EVERYTHING!
                delta = (times[curr] - times[prev])
                x_forecasted = xs[prev] + delta*x_vels[prev]
                y_forecasted = ys[prev] + delta*y_vels[prev]
                b0_forecasted = b0s[prev] + delta*bb0_vels[prev]
                b1_forecasted = b1s[prev] + delta*bb1_vels[prev]
                b2_forecasted = b2s[prev] + delta*bb2_vels[prev]
                b3_forecasted = b3s[prev] + delta*bb3_vels[prev]
                if not isitinfnan(x_forecasted,xs[curr]):
                  x_diffs.append( (x_forecasted - xs[curr])**2 )
                if not isitinfnan(y_forecasted,ys[curr]):
                  y_diffs.append( (y_forecasted - ys[curr])**2 )
                if not isitinfnan(b0_forecasted,b0s[curr]):
                  bb0_diffs.append( (b0_forecasted - b0s[curr])**2 )
                if not isitinfnan(b1_forecasted,b1s[curr]):
                  bb1_diffs.append( (b1_forecasted - b1s[curr])**2 )
                if not isitinfnan(b2_forecasted,b2s[curr]):
                  bb2_diffs.append( (b2_forecasted - b2s[curr])**2 )
                if not isitinfnan(b3_forecasted,b3s[curr]):
                  bb3_diffs.append( (b3_forecasted - b3s[curr])**2 )

  x_mse = sum(x_diffs)/len(x_diffs)
  y_mse = sum(y_diffs)/len(y_diffs)
  print("x MSE is " + str(x_mse))
  print("y MSE is " + str(y_mse))
  print("bb0 MSE is " + str(sum(bb0_diffs)/len(bb0_diffs)))
  print("bb1 MSE is " + str(sum(bb1_diffs)/len(bb1_diffs)))
  print("bb2 MSE is " + str(sum(bb2_diffs)/len(bb2_diffs)))
  print("bb3 MSE is " + str(sum(bb3_diffs)/len(bb3_diffs)))
  return x_mse, y_mse




   TIMESTAMPS         X         Y         Z  ...  v_bb0  v_bb1  v_bb2  v_bb3
0    0.000000  0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0
1    0.000000 -0.135406  0.190364  0.827317  ...    0.0    0.0    0.0    0.0
2    0.000000  0.252479  0.081335  0.280332  ...    0.0    0.0    0.0    0.0
3    0.000000  0.334222  0.142681  0.328718  ...    0.0    0.0    0.0    0.0
4    0.050505  0.001967 -0.000194 -0.001729  ...    0.0    0.0    0.0    0.0

[5 rows x 17 columns]
['TIMESTAMPS', 'X', 'Y', 'Z', 'b0', 'b1', 'b2', 'b3', 'Feat_Index', 'Object_Type', 'Agent_ID', 'v_x', 'v_y', 'v_bb0', 'v_bb1', 'v_bb2', 'v_bb3']


In [26]:
x_mses = []
y_mses=[]
windows= [1,2,3,4]
for i in range(len(windows)):
  x1, y1 = find_MSE_x_y_from_1_frame_ago(windows[i])
  x_mses.append(x1)
  y_mses.append(y1)

print(x_mses)
print(y_mses)


x MSE is 0.010883384468489535
y MSE is 0.00024291566332115775
bb0 MSE is 1607.3266485432466
bb1 MSE is 37.380570657402686
bb2 MSE is 1666.188885298793
bb3 MSE is 41.462469409559446
x MSE is 0.010867649290194095
y MSE is 0.0002427481764245517
bb0 MSE is 1607.3266485432466
bb1 MSE is 37.380570657402686
bb2 MSE is 1666.188885298793
bb3 MSE is 41.462469409559446
x MSE is 0.010867653147596721
y MSE is 0.00024273934099084738
bb0 MSE is 1607.3266485432466
bb1 MSE is 37.380570657402686
bb2 MSE is 1666.188885298793
bb3 MSE is 41.462469409559446
x MSE is 0.010867643572427968
y MSE is 0.0002427576629564739
bb0 MSE is 1607.3266485432466
bb1 MSE is 37.380570657402686
bb2 MSE is 1666.188885298793
bb3 MSE is 41.462469409559446
[0.010883384468489535, 0.010867649290194095, 0.010867653147596721, 0.010867643572427968]
[0.00024291566332115775, 0.0002427481764245517, 0.00024273934099084738, 0.0002427576629564739]


In [0]:
#COPY OF ABOVE CELL: CHECKING FORECAST FROM 2 BEFORE INSTEAD OF 1 BEFORE

x_diffs = []
y_diffs = []
bb0_diffs = []
bb1_diffs = []
bb2_diffs=[]
bb3_diffs=[]
# WAS CONSIDERING DIVIDING UP POS AND NEG ANALAYSES, BUT NEED TRACKING TO WORK WELL REGARDLESS OF THE CASE
for filename in os.listdir(new_save_path):
    file = open(new_save_path+filename, 'rb')
    traj_df = pickle.load(file)
    file.close()
    times = traj_df['TIMESTAMPS']
    xs = traj_df['X']
    ys = traj_df['Y']
    zs = traj_df['Z']
    b0s = traj_df['b0']
    b1s = traj_df['b1']
    b2s = traj_df['b2']
    b3s = traj_df['b3']
    x_vels = traj_df['v_x']
    y_vels = traj_df['v_y']
    bb0_vels = traj_df['v_bb0']
    bb1_vels = traj_df['v_bb1']
    bb2_vels = traj_df['v_bb2']
    bb3_vels = traj_df['v_bb3']
    
    all_ids = traj_df['Agent_ID']
    # NOTE: cannot sort by unique agent IDs, makes more sense to go chronologically
    unique_times, unique_pos = np.unique(times, return_index=True)

    np.append(unique_pos,len(unique_pos)-1)

    for p in range(2,len(unique_pos)-1):
      for curr in range(unique_pos[p],unique_pos[p+1]): 
        curr_id = all_ids[curr]
        for prev in range(unique_pos[p-2],unique_pos[p]):
          if all_ids[prev] == curr_id and xs[prev] >0and ys[prev]>0 and b0s[prev]>0:
            #CALCULATE ESTIMATED EVERYTHING!
            delta = (times[curr] - times[prev])
            x_forecasted = xs[prev] + delta*x_vels[prev]
            y_forecasted = ys[prev] + delta*y_vels[prev]
            b0_forecasted = b0s[prev] + delta*bb0_vels[prev]
            b1_forecasted = b1s[prev] + delta*bb1_vels[prev]
            b2_forecasted = b2s[prev] + delta*bb2_vels[prev]
            b3_forecasted = b3s[prev] + delta*bb3_vels[prev]
            if not isitinfnan(x_forecasted,xs[curr]):
              x_diffs.append( (x_forecasted - xs[curr])**2 )
            if not isitinfnan(y_forecasted,ys[curr]):
              y_diffs.append( (y_forecasted - ys[curr])**2 )
            if not isitinfnan(b0_forecasted,b0s[curr]):
              bb0_diffs.append( (b0_forecasted - b0s[curr])**2 )
            if not isitinfnan(b1_forecasted,b1s[curr]):
              bb1_diffs.append( (b1_forecasted - b1s[curr])**2 )
            if not isitinfnan(b2_forecasted,b2s[curr]):
              bb2_diffs.append( (b2_forecasted - b2s[curr])**2 )
            if not isitinfnan(b3_forecasted,b3s[curr]):
              bb3_diffs.append( (b3_forecasted - b3s[curr])**2 )


print("x MSE is " + str(sum(x_diffs)/len(x_diffs)))
print("y MSE is " + str(sum(y_diffs)/len(y_diffs)))
print("bb0 MSE is " + str(sum(bb0_diffs)/len(bb0_diffs)))
print("bb1 MSE is " + str(sum(bb1_diffs)/len(bb1_diffs)))
print("bb2 MSE is " + str(sum(bb2_diffs)/len(bb2_diffs)))
print("bb3 MSE is " + str(sum(bb3_diffs)/len(bb3_diffs)))




x MSE is 0.012100532672363598
y MSE is 0.0002739192487186796
bb0 MSE is 1789.8377048949737
bb1 MSE is 43.45463991927359
bb2 MSE is 1852.6430078306494
bb3 MSE is 47.267467532569825


TypeError: ignored

In [0]:

num1 = (126, 46)
root = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents'
# path
train_path = root + '/dataset/features/training/' 
test_path = root +  '/dataset/features/testing/' 
modes = ('training', 'testing')
mode_paths = (train_path, test_path)

# depth_np = np.load(depth_complete_paths[vid_num]) # ['distance_map']
# odom_np = np.load(odom_complete_paths[vid_num]) # ['rot', 'trans', 'instrinsic_matrix', 'orientation', 'position']

for mode_iter in range(2):
  num_batches = num1[mode_iter]
  mode = modes[mode_iter]

  cur_path = optimal_crop_path+'/'+mode+'/'
  # feat_path = mode_paths[mode_iter]

  for batch_iter in range(num_batches):
    batch_str = 'batch_%03d.npz'%(batch_iter+1)
    if batch_iter % 15 == 0:
      print(batch_str)

    # cropped_dict = np.load(cur_path+batch_str)
    org_bb = np.load(mode_paths[mode_iter]+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
    resized_bb = np.load(cur_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID

    bounding_box = np.load(optimal_crop_path+mode+'/'+batch_str)
    frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'],mode, num1[mode_iter])
    # odom_paths = retrieve_dir_custom(path_odom, org_bb, mode, num1[mode_iter], split_type=1)
    depth_paths = retrieve_dir_custom(path_depth, org_bb, mode, num1[mode_iter])
    for vid in range(10):
      cur_name = re.split('/',depth_paths[vid])[-1][0:-4]

      ## saving trajectory_df in new location with velocities ##################
      #NO SAVING BEING DONE, trying to use this to read the files!
      if os.path.exists(new_save_path+cur_name+".pkl"):
      # if False:
        pass
      else:
        try:
          file = open(new_save_path+cur_name+'.pkl', 'rb')
          trajectory_df = pickle.load(file)
          file.close()
          # trajectory_df = pd.read_pickle(save_path+cur_name+'.pkl') #????????????????
          print(save_path+cur_name+'.pkl')
          print(trajectory_df.head())
          print(list(trajectory_df.columns.values))

          """
          trajectory_df.reset_index(inplace = True)
          trajectory_df = trajectory_df.drop(['index'], axis = 1)
          trajectory_df['v_x'] = 0.0 # (-6)
          trajectory_df['v_y'] = 0.0 # (-5)
          trajectory_df['v_bb0'] = 0.0 # (-4)
          trajectory_df['v_bb1'] = 0.0 # (-3)
          trajectory_df['v_bb2'] = 0.0 # (-2)
          trajectory_df['v_bb3'] = 0.0 # (-1)
          """
          k_width = 10
          thr = 0.5

          frames = trajectory_df.groupby('Agent_ID')

          # for k in range(1,100):
          #   average_velocity(k, k_width, thr, frames, trajectory_df) # updates trajectory_df for current k

          # trajectory_df.to_pickle(new_save_path+cur_name+".pkl")
          print(new_save_path+cur_name+".pkl")
        except KeyError:
          print('KEY ERROR!!!')
          print(new_save_path+cur_name+".pkl")
      ##########################################################################

batch_001.npz


FileNotFoundError: ignored

In [0]:
npz_save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_1npz/'

root = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents'
# path
train_path = root + '/dataset/features/training/' 
test_path = root +  '/dataset/features/testing/' 
modes = ('training', 'testing')
mode_paths = (train_path, test_path)

num1 = (126, 46)

def retrieve_global_coord(global_df_path, feat_path, batch_str, num_samp, mode='training'):
  glob_feat = np.zeros((10,100, 20, 13))
  feat_index_batch = np.zeros((10, 100, 19))
  org_bb = np.load(feat_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
  frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'], mode, num_samp)
  global_df_paths = retrieve_dir_custom(new_save_path, org_bb, mode, num_samp, split_type=2)
 
  for batch_num in range(10):
    cur_df_path = global_df_paths[batch_num]
    # add try except functionality in case df doesn't exisit
    try:
      trajectory_df = pd.read_pickle(cur_df_path)
      time_stamps = np.linspace(0,5,100) # use time_stamps to retrieve the correct index
      frames = trajectory_df.groupby('TIMESTAMPS')

      for group_name, group_data in frames:
        time_ind = np.argwhere(time_stamps == group_name)[0][0] 
        cur_x, cur_y, cur_z = group_data['X'], group_data['Y'], group_data['Z']
        v_x, v_y = group_data['v_x'], group_data['v_y']
        bb0, bb1, bb2, bb3 = group_data['b0']/128, group_data['b1']/416, group_data['b2']/128, group_data['b3']/416
        if (bb0.any() >1 or bb1.any() >1 or bb2.any()>1 or bb3.any()>1):
          print('Scaled incorrectly!')
        v_bb0, v_bb1, v_bb2, v_bb3 = group_data['v_bb0']/128, group_data['v_bb1']/416, group_data['v_bb2']/128, group_data['v_bb3']/416
        

        glob_feat[batch_num, time_ind,0:len(cur_x),0] = cur_x
        glob_feat[batch_num, time_ind,0:len(cur_x),1] = cur_y
        glob_feat[batch_num, time_ind,0:len(cur_x),2] = cur_z
        glob_feat[batch_num, time_ind,0:len(cur_x),3] = v_x
        glob_feat[batch_num, time_ind,0:len(cur_x),4] = v_y
        glob_feat[batch_num, time_ind,0:len(cur_x),5] = bb0
        glob_feat[batch_num, time_ind,0:len(cur_x),6] = bb1
        glob_feat[batch_num, time_ind,0:len(cur_x),7] = bb2
        glob_feat[batch_num, time_ind,0:len(cur_x),8] = bb3
        glob_feat[batch_num, time_ind,0:len(cur_x),9] = v_bb0
        glob_feat[batch_num, time_ind,0:len(cur_x),10] = v_bb1
        glob_feat[batch_num, time_ind,0:len(cur_x),11] = v_bb2
        glob_feat[batch_num, time_ind,0:len(cur_x),12] = v_bb3


        feat_index_batch[batch_num, time_ind,0:len(cur_x)] = group_data['Feat_Index']

    except:
      print("Dataframe doesn't exist: " + str(cur_df_path))
  glob_feat = take_out_nans(glob_feat)

  # modifications
  glob_feat[glob_feat == np.inf] = 0
  glob_feat[glob_feat*(-1) == np.inf] = 0
  glob_feat[:,:,:,0:5] = glob_feat[:,:,:,0:5] #*0.1  ## change/normalize the global features

  return glob_feat, feat_index_batch

def take_out_nans(phys_feat):
    if np.sum(np.isnan(phys_feat)) > 0:
      nan_indices = np.argwhere(np.isnan(phys_feat))
      for k in range(nan_indices.shape[0]):
        phys_feat[nan_indices[k,0],nan_indices[k,1],nan_indices[k,2],nan_indices[k,3]] = 0
    return phys_feat

In [0]:
#WTF IS THIS FOR?

mode_paths = (train_path, test_path)
modes = ('training', 'testing')

for cur_mod in range(2):
  cur_num = num1[cur_mod]
  n_batchs = np.arange(1,cur_num+1)

  count_batch = 0  
  start_time = time.time()
  for batch in n_batchs:
      print(batch)
      count_batch += 1       

      file_name = '%03d' %batch
      batch_str = 'batch_'+file_name+'.npz'

      cur_save_path = npz_save_path+modes[cur_mod]+'/'+batch_str
    #  print(file_name)
      batch_data = np.load(mode_paths[cur_mod]+batch_str, allow_pickle = True)
      batch_xs = batch_data['data']
      batch_ys = batch_data['labels']

      df_paths = retrieve_dir_custom(new_save_path, batch_data, modes[cur_mod], cur_num, split_type=2)

      phys_feat, feat_ind = retrieve_global_coord(df_paths, mode_paths[cur_mod], batch_str, cur_num, mode=modes[cur_mod])

      # if os.path.exists(save_path+cur_name+".npz"):
      if False:
        pass
      else:
        np.savez(cur_save_path ,phys_feat = phys_feat, feat_ind = feat_ind)

      ## form path, and save phys_feat and feat_ind

1
2
3
4
5
6
7
8
9
10
11
12
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_5v/negative_000027.pkl
13
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_5v/negative_000037.pkl
14
15
16
17
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_5v/negative_000120.pkl
18
19
20
21
22
23
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_5v/negative_000113.pkl
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_5v/negative_000322.pkl
Dataframe doesn't exist: /content/gdrive/My Drive/Accident_Anticipatio

In [0]:
print(phys_feat.shape)
print(np.max(phys_feat))

# phys_feat[:,:,:,[3,4,9,10,11,12]] = np.clip(phys_feat[:,:,:,[3,4,9,10,11,12]],-5,5)

phys_feat, feat_ind = phys_dict['phys_feat'], phys_dict['feat_ind'] 

(10, 100, 20, 13)
3.25


In [0]:
print(phys_feat[1,5,:,2])
print(phys_feat[1,12+5,:,2])
print(phys_feat[1,5,:,4]*12+phys_feat[1,5,:,2])
print(phys_feat[1,25+5,:,0])
print(phys_feat[1,5,:,4]*25+phys_feat[1,5,:,2])
print(phys_feat[1,50+5,:,0])
print(phys_feat[1,5,:,4]*50+phys_feat[1,5,:,2])


[0.00574861 0.49815479 0.55641547 0.5650103  0.46162756 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
[0.04277584 0.55806106 0.54956918 0.51162326 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
[0.01803061 0.48581987 0.58396008 0.68763021 0.47982803 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
[-0.00690506 -0.18378701 -0.37795557  0.03164489  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
[0.03133612 0.47245703 0.61380007 0.82046844 0.4995452  0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.

# Adding future positions

In [0]:
## function to save velocities, repickling/saving pandas array.
file = open(save_path+pkl_str, 'rb')
trajectory_df = pickle.load(file)
file.close()

trajectory_df.reset_index(inplace = True)
trajectory_df = trajectory_df.drop(['index'], axis = 1)
trajectory_df['v_x'] = 0.0
trajectory_df['v_y'] = 0.0

k_width = 10
thr = 0.5

frames = trajectory_df.groupby('Agent_ID')
for k in range(1,100):
  average_velocity(k, k_width, thr, frames, trajectory_df) # updates trajectory_df for current k


0.0 0.0
    TIMESTAMPS         X         Y         Z  Object_Type  Agent_ID  v_x  v_y
0     0.000000  0.381186  0.013228  0.510461          1.0       1.0  0.0  0.0
1     0.000000 -0.184158  0.219678  1.304488          4.0       5.0  0.0  0.0
2     0.000000 -0.711506  0.182028  0.712492          4.0       6.0  0.0  0.0
3     0.000000  0.000000  0.000000  0.000000          0.0       0.0  0.0  0.0
4     0.050505  0.321307 -0.018012  0.489319          1.0       1.0  0.0  0.0
5     0.050505 -0.310028  0.165271  2.531496          4.0       4.0  0.0  0.0
6     0.050505  0.211735  0.100965  0.371153          3.0       3.0  0.0  0.0
7     0.050505 -0.739551 -0.041960  2.390344          4.0       5.0  0.0  0.0
8     0.050505 -0.928212  0.174489  0.902489          4.0       6.0  0.0  0.0
9     0.050505  0.531836 -0.093157  3.976911          3.0       2.0  0.0  0.0
10    0.050505  0.000027  0.000033 -0.000421          0.0       0.0  0.0  0.0
11    0.101010  0.311525 -0.037696  0.508877          1.

In [0]:
# Plotting regulat bounding boxes
